In [2]:
# ! pip install xarray

  Obtaining dependency information for xarray from https://files.pythonhosted.org/packages/1a/66/39d1df58fc6f06c19551dd9c8da5f9504b424b7f7212424fe25e69f97149/xarray-2023.10.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.1 MB/s eta 0:00:00a 0:00:01


In [3]:
import xarray as xr
import netCDF4 as nc
import os
import pandas as pd

In [4]:


# Define the base directory where NetCDF files are located
# base_directory = "/Users/rd/Desktop/Data/"  # Modify this to the appropriate base directory
base_directory = "/Users/gaylynruvere/Documents/Machine_Learning/Hackathons/UN_Hackathon_2023_Peat_Fires/"  # Modify this to the appropriate base directory

# Define the variables you want to extract
# Define the variables you want to extract
desired_variables = ["T2M_MIN", "T2M_MAX", "T2M", "PRECTOTCORR", "QV2M", "WS10M", "WD10M", "WS10M_MAX", "WS10M_MIN"]  # Replace with your desired variable names
min_latitude = -6.92276
max_latitude = 7.10208
min_longitude = 94.71447
max_longitude = 107.28015
# Define the years and months to loop through
years = ["2010", "2011"]
months = [str(i).zfill(2) for i in range(1, 3)]
days = [str(i).zfill(2) for i in range(1, 32)]

# Create an empty list to store DataFrame objects
dataframes = []

# Loop through the years, months, and days
for year in years:
    print(year)
    for month in months:
        for day in days:
            # Construct the directory path based on the year, month, and day
            directory_path = os.path.join(base_directory, year, month)
            print(directory_path)

            # Check if the directory exists
            if not os.path.exists(directory_path):
                continue

            # Construct the file name based on the year, month, and day
            file_name = f"power_901_daily_{year}{month}{day}_merra2_lst.nc"
            file_path = os.path.join(directory_path, file_name)
            # print(file_path)

            # Check if the file exists
            if not os.path.exists(file_path):
                continue  # File does not exist, continue to the next day

            # Open the NetCDF file
            dataset = xr.open_dataset(file_path)

            df = dataset.to_dataframe()
            #print(df)

            # Close the NetCDF file
            dataset.close()

            # Keep only the desired variables in the DataFrame
            df = df[desired_variables]
            df = df.reset_index()
            
            # Example of conditions for latitude and longitude
            #lat_condition = (df['lat'].between(-6.92276, 7.10208, inclusive=True))
            #long_condition = (df['long'].between(94.71447, 107.28015, inclusive=True))
            #if (lat_condition & long_condition).any():
                 #dataframes.append(df)
            df = df[(df['lat'] >= min_latitude) & (df['lat'] <= max_latitude) & (df['lon'] >= min_longitude) & (df['lon'] <= max_longitude)]
            print(df)
            dataframes.append(df)

# Concatenate all DataFrames into one
final_df = pd.concat(dataframes, ignore_index=True)

# Print the final DataFrame
#print(final_df)


2010
/Users/gaylynruvere/Documents/Machine_Learning/Hackathons/UN_Hackathon_2023_Peat_Fires/2010/01
            lon  lat       time     T2M_MIN     T2M_MAX         T2M  \
159007   95.000 -6.5 2010-01-01  301.351562  301.679688  301.507812   
159008   95.000 -6.0 2010-01-01  301.421875  301.757812  301.648438   
159009   95.000 -5.5 2010-01-01  301.515625  302.117188  301.828125   
159010   95.000 -5.0 2010-01-01  301.601562  302.359375  301.953125   
159011   95.000 -4.5 2010-01-01  301.695312  302.437500  302.085938   
...         ...  ...        ...         ...         ...         ...   
165889  106.875  5.0 2010-01-01  299.273438  299.656250  299.554688   
165890  106.875  5.5 2010-01-01  299.343750  299.578125  299.468750   
165891  106.875  6.0 2010-01-01  299.281250  299.507812  299.406250   
165892  106.875  6.5 2010-01-01  299.320312  299.453125  299.382812   
165893  106.875  7.0 2010-01-01  299.242188  299.437500  299.343750   

        PRECTOTCORR      QV2M      WS10M     WD

In [7]:
final_df.to_csv('Climate_2011_2012.csv')